**Imports**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense, Dropout, Input, Flatten
from keras.models import Sequential
from keras.activations import linear, softmax, relu